# 언어모델링
- 어린왕자 소설 데이터 사용
- 음절 단위 언어 모델링

In [1]:
from keras.layers.recurrent import SimpleRNN
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np

Using TensorFlow backend.
/Users/hwa-kim/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/hwa-kim/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/hwa-kim/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/hwa-kim/opt/anaconda3/lib/pytho

In [2]:
!curl https://raw.githubusercontent.com/StillWork/data/master/The_little_prince.txt\
    -o The_little_prince.txt
fin = open("The_little_prince.txt", 'rb')
lines = []
for line in fin:
    line = line.strip().lower()
    line = line.decode("euckr", "ignore")
    if len(line) == 0:
        continue
    lines.append(line)
fin.close()
text = " ".join(lines)

# char indexing
chars = set([c for c in text])
nb_chars = len(chars)
char2index = dict((c, i) for i, c in enumerate(chars))
index2char = dict((i, c) for i, c in enumerate(chars))

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 76620  100 76620    0     0  24949      0  0:00:03  0:00:03 --:--:-- 24949:--:--  0:00:02 --:--:--     0


In [3]:
text

'여섯 살 적에 나는 "체험한 이야기"라는 제목의, 원시림에 관한 책에서 기막힌 그림 하나를 본 적이 있다. 맹수를 집어삼키고 있는 보아 구렁이 그림이었다. 위의 그림은 그것을 옮겨 그린 것이다. 그 책에는 이렇게 씌어 있었다. "보아 구렁이는 먹이를 씹지도 않고 통째로 집어삼킨다.그리고는 꼼짝도 하지 못하고 여섯 달 동안 잠을 자면서 그것을 소화시킨다." 나는 그래서 밀림 속에서의 모험에 대해 한참 생각해 보고 난 끝에 색연필을 가지고 내 나름대로 내 생애 첫번째 그림을 그려보았다. 나의 그림 제 1호였다. 그것은 이런 그림이었다. 나는 그 걸작품을 어른들에게 보여 주면서내 그림이 무섭지 않느냐고 물었다. 그들은 "모자가 뭐가 무섭다는 거니?" 하고 대답했다. 내 그림은 모자를 그린 게 아니었다. 그것은 코끼리를 소화시키고 있는 보아 구렁이었다. 그래서 나는 어른들이 알아볼 수 있도록 보아 구렁이의 속을 그렸다. 어른들은 언제나 설명을 해주어야만 한다. 나의 그림 제 2호는 이러했다. 어른들은 속이 보이거나 보이지 않거나 하는 보아 구렁이의 그림들은 집어치우고 차라리 지리, 역사, 계산, 그리고 문법 쪽에 관심을 가져보는 게 좋을 것이라고 충고해 주었다. 그래서 나는 여섯 살 적에 화가라는 멋진 직업을 포기해 버렸다.내 그림제 1호와 제 2호가 성공을 거두지 못한 데 낙심해 버렸던 것이다. 어른들은언제나 스스로는 아무것도 이해하지 못한다.자꾸자꾸 설명을 해주어야 하니 맥빠지는 노릇이 아닐 수 없다. 그래서 다른 직업을 선택하지 않을 수 없게 된 나는 비행기 조종하는 법을배웠다.세계의 여기저기 거의 안 가본 데 없이 나는 날아다녔다.그러니지리는 정말로 많은 도움을 준 셈이었다.한번 슬쩍 보고도 중국과 애리조나를 나는 구별할 수 있었던 것이다.그것은 밤에 길을 잃었을 때 아주 유용한 일이다. 나는 그리하여 일생 동안 수없이 많은 점잖은 사람들과수많은 접촉을 가져왔다.어른들 틈에서 많이 살아온 것이다.나는 가까이서 그들을 볼 수있었다. 그렇다고 해서 그들에 대한 

In [0]:
print(index2char[0])
print(char2index["가"])

태
645


In [0]:
for i in range(10):
    print(index2char[i])

태
과
상
적
밝
살
완
분
접
까


In [0]:
text[:100]

'여섯 살 적에 나는 "체험한 이야기"라는 제목의, 원시림에 관한 책에서 기막힌 그림 하나를 본 적이 있다. 맹수를 집어삼키고 있는 보아 구렁이 그림이었다. 위의 그림은 그것을 옮겨'

In [0]:
# X, Y 데이터 생성
# "The sky was falling"
# 아래와 같이 10개 문자를 주면 다음 문자가 정답
#   The sky wa -> s
#   he sky was ->  
#   e sky was  -> f
#    sky was f -> a
#   sky was fa -> l

print("Creating input and label text...")
SEQLEN = 10
STEP = 1

input_chars = []
label_chars = []
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i:i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

Creating input and label text...


In [0]:
nb_chars

850

In [0]:
# 데이터 벡터화
# 원핫 인코딩 - True False
# 주어진 어린왕자 데이터에는 총 850개의 글자 포함
# input_chars(44792)에서 각 row마다 10개의 글자를 벡터화(850) => (44792, 10, 850)
# 전체 글자에 대해 해당 되는 글자에만 True, 나머지는 False
#
print("Vectorizing input and label text...")
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)

for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1
X.shape

Vectorizing input and label text...


(44792, 10, 850)

In [0]:
# np.argmax(X[:3])
np.argmax(X[0,0,:500])

163

In [0]:
# RNN 모델
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 10

model = Sequential()
model.add(SimpleRNN(HIDDEN_SIZE, return_sequences=False,
                    input_shape=(SEQLEN, nb_chars),
                    unroll=True))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [0]:
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    
    # 각 학습마다 테스트 데이터( 연속된 10개 글자 )를 부여하고
    # 다음 문자 10개를 예측
    
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    print("입력: %s" % (test_chars))
    print(test_chars, end="")
    
    for i in range(NUM_PREDS_PER_EPOCH):
        Xtest = np.zeros((1, SEQLEN, nb_chars))
        for i, ch in enumerate(test_chars):
            Xtest[0, i, char2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2char[np.argmax(pred)]
        print(ypred, end="")
        
        # move forward with test_chars + ypred
        test_chars = test_chars[1:] + ypred
    print()

Iteration #: 0
Epoch 1/1
44792/44792 [==============================] - 15s 342us/step - loss: 4.2484
입력:  무심히 내버 려두
 무심히 내버 려두 그는 그는 그는 
Iteration #: 1
Epoch 1/1
44792/44792 [==============================] - 15s 342us/step - loss: 3.6692
입력:  계속했다. "아저
 계속했다. "아저 그는 그 이 있는
Iteration #: 2
Epoch 1/1
44792/44792 [==============================] - 16s 349us/step - loss: 3.4196
입력: 고 있는지를 알고 
고 있는지를 알고 있는 것이 다. 그
Iteration #: 3
Epoch 1/1
44792/44792 [==============================] - 14s 321us/step - loss: 3.2344
입력: 다. 여섯 살 적에
다. 여섯 살 적에 있었다. 그러자 
Iteration #: 4
Epoch 1/1
44792/44792 [==============================] - 15s 326us/step - loss: 3.0832
입력:  가지고 뭘 해?"
 가지고 뭘 해?" "그래." " 어
Iteration #: 5
Epoch 1/1
44792/44792 [==============================] - 15s 340us/step - loss: 2.9579
입력: 통 별이 아니었다!
통 별이 아니었다! 그러나 그는 그 
Iteration #: 6
Epoch 1/1
44792/44792 [==============================] - 15s 335us/step - loss: 2.8488
입력: 들어올리며 다시 답
들어올리며 다시 답했다. "그래서 그
Iteration #: 7
Epoch 1/1
44792/447